### PROCESO DE CARGA

Una vez que tenemos todas las tablas que necesitamos, vamos a realizar el proceso de carga. Las tablas que vamos a cargar en nuestra base de datos de DBeaver son:

- 01.Empresas_IBEX35: Nombre de la empresa, ticker, sector y subsector
- 02.01.ratios_ticker_año_IBEX35: Ticker, Año, PER, ROE, EV/EBIT, Dividend_Yiend, deuda_financiera/EBIT
- 02.02.ratios_ticker_IBEX35: Ticker, Beta
- 03.Cotizaciones_IBEX35: Ticker, Fecha, Close, Volume, Variación, Año
- 04.Dividendos_IBEX35: Ticker, Año, Dividendos, Precio_Cierre
- 05.cuentas_pyg_IBEX35_limpio: Ticker, Año, Total Revenue, Net Income

Todas las tablas están relacionadas por la columna Ticker por lo que se trata de una base de datos en forma de copo de nieve en la que la tabla 01.Empresas_IBEX35 está en el centro y de la que cuelgan el resto de tablas.


In [1]:
#Primero importamos las librerias necesarias pàra el trabajo
import pandas as pd
import numpy as np 
import psycopg2 as ps


In [27]:
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()

In [7]:
#Comprobamos la conexión
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Este código solo se puede ejecutar una vez

In [33]:
#Creamos en la base de datos la tabla "01.Empresas_IBEX35" para almacenar los datos
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS "empresas_ibex35" (
    "ID" SERIAL PRIMARY KEY,
    "Nombre" VARCHAR(255) NOT NULL,
    "Ticker" VARCHAR(10) NOT NULL,
    "Sector" VARCHAR(255) NOT NULL,
    "Subsector" VARCHAR(255) NOT NULL
);
""")
#Commit los cambios
conn.commit()
#cerramos la conexión
cur.close()
conn.close()

In [35]:
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Comprobamos que se ha creado la tabla
cur.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
""")
print(cur.fetchall())

#Cerramos la conexión
cur.close()
conn.close()

[('empresas_ibex35',)]


In [2]:
#Importamos los datos del csv 01.Empresas_IBEX35 para analizarlo y después cargarlo en la base de datos
df_01_empresas = pd.read_csv('data/01.Empresas_IBEX35.csv', sep=',')
df_01_empresas


,Name,Ticker,Sector,Subsector
0,Grifols S.A.,GRF.MC,Healthcare,Drug Manufacturers - General
1,Indra A,IDR.MC,Technology,Information Technology Services
2,Caixabank SA,CABK.MC,Financial Services,Banks - Regional
3,Repsol,REP.MC,Energy,Oil & Gas Integrated
4,Telefonica,TEF.MC,Communication Services,Telecom Services
5,Bankinter,BKT.MC,Financial Services,Banks - Regional
6,Amadeus IT Group S.A.,AMS.MC,Technology,Information Technology Services
7,Mapfre,MAP.MC,Financial Services,Insurance - Diversified
8,Industria de Diseno Textil SA,ITX.MC,Consumer Cyclical,Apparel Retail
9,Banco de Sabadell S.A,SAB.MC,Financial Services,Banks - Diversified


In [3]:
df_01_empresas.dtypes

Name         object
Ticker       object
Sector       object
Subsector    object
dtype: object

In [37]:
#Ya sabemos que los tipos de datos son correctos, por lo que no es necesario hacer ninguna conversión
#Creamos la conexión a la base de datos
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Ahora vamos a cargar los datos en la tabla "01.Empresas_IBEX35" de la base de datos
for index, row in df_01_empresas.iterrows():
    cur.execute("""
    INSERT INTO "empresas_ibex35" ("Nombre", "Ticker", "Sector", "Subsector")
    VALUES (%s, %s, %s, %s);
    """, (row['Name'], row['Ticker'], row['Sector'], row['Subsector']))
    #Commit los cambios
    conn.commit()

#cerramos la conexión
cur.close()
conn.close()


In [38]:
#Comprobamos que se han cargado los datos en la tabla
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Comprobamos que se han cargado los datos en la tabla
cur.execute("""
SELECT * FROM "empresas_ibex35";
""")
#Guardamos los resultados en un dataframe
df_empresas = pd.DataFrame(cur.fetchall(), columns=['ID', 'Nombre', 'Ticker', 'Sector', 'Subsector'])
#Mostramos los resultados
df_empresas

,ID,Nombre,Ticker,Sector,Subsector
0,1,Grifols S.A.,GRF.MC,Healthcare,Drug Manufacturers - General
1,2,Indra A,IDR.MC,Technology,Information Technology Services
2,3,Caixabank SA,CABK.MC,Financial Services,Banks - Regional
3,4,Repsol,REP.MC,Energy,Oil & Gas Integrated
4,5,Telefonica,TEF.MC,Communication Services,Telecom Services
5,6,Bankinter,BKT.MC,Financial Services,Banks - Regional
6,7,Amadeus IT Group S.A.,AMS.MC,Technology,Information Technology Services
7,8,Mapfre,MAP.MC,Financial Services,Insurance - Diversified
8,9,Industria de Diseno Textil SA,ITX.MC,Consumer Cyclical,Apparel Retail
9,10,Banco de Sabadell S.A,SAB.MC,Financial Services,Banks - Diversified


In [42]:
#Como ya vimos anteriormente en la obtención de los datos, a la tabla le faltan los datos de la empresa "PUIG SA", por lo que vamos a añadirlos
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Ahora vamos a cargar los datos en la tabla "empresas_ibex35" de la base de datos
cur.execute("""
            INSERT INTO "empresas_ibex35" ("Nombre", "Ticker", "Sector", "Subsector")
            VALUES ('PUIG Brands','PUIG.MC', 'Specialty Retailers', 'Consumer Cyclicals')
            """)

#Commit los cambios
conn.commit()

#cerramos la conexión
cur.close()
conn.close()


In [23]:
#Cargamos los datos del csv 02.01.ratios_ticker_año_IBEX35 y los analizamos antes de crear la tabla
df_02_ratios = pd.read_csv('data/02.01.ratios_ticker_año_IBEX35.csv', sep=',')
df_02_ratios.dtypes

ticker                    object
Año                        int64
PER                      float64
ROE                      float64
EV/EBIT                  float64
Dividend_Yield           float64
deuda_financiera/EBIT    float64
dtype: object

In [24]:
df_02_ratios.isnull().sum()

ticker                   0
Año                      0
PER                      0
ROE                      0
EV/EBIT                  0
Dividend_Yield           0
deuda_financiera/EBIT    0
dtype: int64

In [6]:
#los tipos de datos son correctos, por lo que no es necesario hacer ninguna conversión asique vamos a crear la tabla ratios_ticker_año_IBEX35
#Conectamos a la base de datos
conn = ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Creamos la tabla ratios_ticker_año_IBEX35
cur.execute("""
CREATE TABLE IF NOT EXISTS "ratios_ticker_año_IBEX35" (
    "ID" SERIAL PRIMARY KEY,
    "Ticker" VARCHAR(10) NOT NULL,
    "Año" INT NOT NULL,
    "PER" FLOAT,
    "ROE" FLOAT,
    "EV_EBIT" FLOAT,
    "Dividend_yield" FLOAT,
    "Deuda_finan/EBIT" FLOAT
);
""")
#Commit los cambios
conn.commit()
#cerramos la conexión
cur.close()
conn.close()

In [25]:
#Cargamos los datos del dataframe df_02_ratios en la tabla ratios_ticker_año_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Primero vamos a eliminar todos los datos de la tabla "ratios_ticker_año_IBEX35" para evitar duplicados
cur.execute("""
DELETE FROM "ratios_ticker_año_IBEX35";
""")
#Commit los cambios
conn.commit()
#Ahora vamos a cargar los datos en la tabla "ratios_ticker_año_IBEX35" de la base de datos
for index, row in df_02_ratios.iterrows():
    cur.execute("""
    INSERT INTO "ratios_ticker_año_IBEX35" ("Ticker", "Año", "PER", "ROE", "EV_EBIT", "Dividend_yield", "Deuda_finan/EBIT")
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, (row['ticker'], row['Año'], row['PER'], row['ROE'], row['EV/EBIT'], row['Dividend_Yield'], row['deuda_financiera/EBIT']))
    #Commit los cambios
    conn.commit()
#cerramos la conexión
cur.close()
conn.close()

In [11]:
#comprobamos que se han cargado los datos en la tabla
#Cargamos los datos del dataframe df_02_ratios en la tabla ratios_ticker_año_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Comprobamos que se han cargado los datos en la tabla
cur.execute("""
SELECT * FROM "ratios_ticker_año_IBEX35";
""")
#Guardamos los resultados en un dataframe
df_ratios = pd.DataFrame(cur.fetchall(), columns=['ID', 'Ticker', 'Año', 'PER', 'ROE', 'EV_EBIT', 'Dividend_yield', 'Deuda_finan/EBIT'])
#Mostramos los resultados
df_ratios

,ID,Ticker,Año,PER,ROE,EV_EBIT,Dividend_yield,Deuda_finan/EBIT
0,1,ACS.MC,2020,6.763836,0.253948,21.638370,0.0987,23.209320
1,2,ACS.MC,2021,-7.718406,-0.115541,151.199289,0.0915,195.413346
2,3,ACS.MC,2022,9.477766,0.120458,20.199583,0.0858,23.057017
3,4,ACS.MC,2023,12.512750,0.146380,18.219412,0.0525,17.719905
4,5,ACS.MC,2024,14.950232,0.175550,15.725971,0.0421,15.203906
...,...,...,...,...,...,...,...,...
140,141,TEF.MC,2024,-456.096812,-0.002533,23.869408,0.0761,19.850633
141,142,UNI.MC,2021,1.666680,0.175983,90.201863,0.0143,109.779866
142,143,UNI.MC,2022,8.224456,0.042945,281.395171,0.0233,288.829624
143,144,UNI.MC,2023,7.755769,0.040144,302.784508,0.0641,324.208845


In [12]:
#Cargamos en un dataframe los datos de la tabla 02.02.ratios_ticker_IBEX35 y los analizamos
df_02_02_ratios_ticker = pd.read_csv('data/02.02.ratios_ticker_IBEX35.csv', sep=',')
df_02_02_ratios_ticker.dtypes

ticker     object
beta      float64
dtype: object

In [13]:
#los tipos de datos son correctos, por lo que no es necesario hacer ninguna conversión asique valos a crear la tabla ratios_beta_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Creamos la tabla ratios_beta_IBEX35
cur.execute("""
CREATE TABLE IF NOT EXISTS "ratios_beta_IBEX35" (
    "Ticker" VARCHAR(10) NOT NULL,
    "beta" FLOAT
);
""")
#Commit los cambios
conn.commit()
#cerramos la conexión	
cur.close()
conn.close()

In [14]:
#Cargamos los datos del data frame df_02_02_ratios_ticker en la tabla ratios_beta_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#cargamos los datos del dataframe
for index, row in df_02_02_ratios_ticker.iterrows():
    cur.execute("""
    INSERT INTO "ratios_beta_IBEX35" ("Ticker", "beta")
    VALUES (%s, %s);
    """, (row['ticker'], row['beta']))
    #Commit los cambios
    conn.commit()
    #cerramos la conexión
cur.close()
conn.close()

In [15]:
#comprobamos que se han cargado los datos en la tabla
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
cur.execute("""
SELECT* FROM "ratios_beta_IBEX35";
            """)
#Guardamos los resultados en un dataframe
df_beta = pd.DataFrame(cur.fetchall(), columns=['Ticker', 'beta'])
#Mostramos los resultados
df_beta

,Ticker,beta
0,GRF.MC,0.7062
1,IDR.MC,0.8588
2,CABK.MC,1.2257
3,REP.MC,1.0146
4,TEF.MC,0.8370
5,BKT.MC,1.1781
6,AMS.MC,1.1865
7,MAP.MC,0.9444
8,ITX.MC,1.0116
9,SAB.MC,1.6560


In [30]:
#Cargamos los datos del csv 03.Cotizaciones_IBEX35 y los analizamos antes de crear la tabla
df_03_cotizaciones = pd.read_csv('data/03.Cotizaciones_IBEX35.csv', sep=',')
df_03_cotizaciones.dtypes

ticker        object
Date          object
Close        float64
Volume         int64
Variación    float64
Año            int64
dtype: object

In [33]:
#Vemos en excel que en la colunma fecha existen dos cosas que nos pueden dar problemas, primero vamos a eliminar todos los " 00:00:00+02:00" y " 00:00:00+01:00" qu son segundos y zona horaria
#despues vamos a cambiar los - por /
df_03_cotizaciones['Date'] = df_03_cotizaciones['Date'].str.replace(' 00:00:00+02:00', '')
df_03_cotizaciones['Date'] = df_03_cotizaciones['Date'].str.replace(' 00:00:00+01:00', '')
df_03_cotizaciones['Date'] = df_03_cotizaciones['Date'].str.replace('-', '/')
#Ahora vamos a cambiar el tipo de dato de la columna fecha a datetime
df_03_cotizaciones['Date'] = pd.to_datetime(df_03_cotizaciones['Date'], format='%Y/%m/%d')
df_03_cotizaciones.dtypes

ticker               object
Date         datetime64[ns]
Close               float64
Volume                int64
Variación           float64
Año                   int64
dtype: object

In [35]:
#Todos los tipos de datos son correctos, por lo que no es necesario hacer ninguna conversión asique vamos a crear la tabla cotizaciones_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#creamos la tabla cotizaciones_IBEX35
cur.execute("""
CREATE TABLE IF NOT EXISTS "Cotizaciones_IBEX35"
            (
    "Ticker" VARCHAR(10) NOT NULL,
    "Fecha" DATE NOT NULL,
    "Cierre" FLOAT,
    "Volumen" FLOAT,
    "Variación" FLOAT,
    "Año" INT
);
""")
#Commit los cambios
conn.commit()
#cerramos la conexión
cur.close()
conn.close()

In [36]:
#Ahora cargamos los datos del dataframe df_03_cotizaciones en la tabla cotizaciones_IBEX35º
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#cargamos los datos
for index, row in df_03_cotizaciones.iterrows():
    cur.execute("""
    INSERT INTO "Cotizaciones_IBEX35" ("Ticker", "Fecha", "Cierre", "Volumen", "Variación", "Año")
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['ticker'], row['Date'], row['Close'], row['Volume'], row['Variación'], row['Año']))
    #Commit los cambios
    conn.commit()

#cerramos la conexión
cur.close()
conn.close()


In [37]:
#Comprobamos que se han cargado los datos en la tabla
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
cur.execute("""
SELECT* FROM "Cotizaciones_IBEX35";
            """)
df_cotiz = pd.DataFrame(cur.fetchall(), columns=['Ticker', 'Fecha', 'Cierre', 'Volumen', 'Variación', 'Año'])
#Mostramos los resultados
df_cotiz

,Ticker,Fecha,Cierre,Volumen,Variación,Año
0,GRF.MC,2020-01-02,30.99,392672.0,NaN,2020
1,GRF.MC,2020-01-03,31.02,641127.0,0.0009,2020
2,GRF.MC,2020-01-06,31.01,386910.0,-0.0003,2020
3,GRF.MC,2020-01-07,31.86,2090693.0,0.0274,2020
4,GRF.MC,2020-01-08,31.83,609008.0,-0.0009,2020
...,...,...,...,...,...,...
47640,PUIG.MC,2025-04-25,16.28,484878.0,0.0106,2025
47641,PUIG.MC,2025-04-28,16.56,555283.0,0.0172,2025
47642,PUIG.MC,2025-04-29,16.69,878013.0,0.0079,2025
47643,PUIG.MC,2025-04-30,16.43,404284.0,-0.0156,2025


In [4]:
#Cargamos los datos del csv 04.Dividendos_IBEX35 y los analizamos antes de crear la tabla
df_04_dividendos = pd.read_csv('data/04.Dividendos_IBEX35.csv', sep=',')
df_04_dividendos.dtypes

ticker                object
Año                    int64
Dividendos           float64
Precio_Cierre_Año    float64
Dividend_Yield       float64
dtype: object

In [2]:
#Todos los tipos de datos son correctos, por lo que no es necesario hacer ninguna conversión asique vamos a crear la tabla dividendos_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Creamos la tabla dividendos_IBEX35
cur.execute("""
CREATE TABLE IF NOT EXISTS "Dividendos_IBEX35"
            (
    "Ticker" VARCHAR(10) NOT NULL,
    "Año" INT,
    "Dividendo" FLOAT,
    "Precio_cierre_año" FLOAT,
    "Dividendo_yield" FLOAT
);
""")
#Commit los cambios
conn.commit()
cur.close()
conn.close()

In [8]:
#Cargamos los datos del dataframe df_04_dividendos en la tabla dividendos_IBEX35
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
for index, row in df_04_dividendos.iterrows():
    cur.execute("""
    INSERT INTO "Dividendos_IBEX35" ("Ticker", "Año", "Dividendo", "Precio_cierre_año", "Dividendo_yield")
    VALUES (%s, %s, %s, %s, %s);
    """, (row['ticker'], row['Año'], row['Dividendos'], row['Precio_Cierre_Año'], row['Dividend_Yield']))
    #Commit los cambios
    conn.commit()

#cerramos la conexión
cur.close()
conn.close()

In [10]:
#Comprobamos que se han cargado los datos en la tabla
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
cur.execute("""
SELECT* FROM "Dividendos_IBEX35";
            """)
df_div = pd.DataFrame(cur.fetchall(), columns=["Ticker", "Año", "Dividendo", "Precio_cierre_año", "Dividendo_yield"])
#Mostramos los resultados
df_div

,Ticker,Año,Dividendo,Precio_cierre_año,Dividendo_yield
0,ACS.MC,2020,1.99,20.16,0.0987
1,ACS.MC,2021,1.72,18.79,0.0915
2,ACS.MC,2022,2.00,23.32,0.0858
3,ACS.MC,2023,1.96,37.36,0.0525
4,ACS.MC,2024,2.02,47.99,0.0421
...,...,...,...,...,...
159,UNI.MC,2021,0.01,0.70,0.0143
160,UNI.MC,2022,0.02,0.86,0.0233
161,UNI.MC,2023,0.05,0.78,0.0641
162,UNI.MC,2024,0.11,1.22,0.0902


In [11]:
#Abrimos el csv 05.cuentas_pyg_IBEX35_limpio y comprobamos que los tipos de datos son correctos
df_05_cuentas_pyg = pd.read_csv('data/05.cuentas_pyg_IBEX35_limpio.csv', sep=',')
df_05_cuentas_pyg.dtypes

ticker           object
Año               int64
totalRevenue    float64
netIncome       float64
dtype: object

In [16]:
#Creamos la tabla cuentas_pyg_IBEX35
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
#Creamos la tabla cuentas_pyg_IBEX35 con las columnas ticker, año, ventas y beneficio_neto
cur.execute("""
CREATE TABLE IF NOT EXISTS "Cuentas_PYG_IBEX35"
            (
    "Ticker" VARCHAR(10) NOT NULL,
    "Año" INT,
    "Ventas" FLOAT,
    "Beneficio_neto" FLOAT
);
""")
#Commit los cambios
conn.commit()
cur.close()
conn.close()

In [19]:
#cargamos los datos del dataframe df_05_cuentas_pyg en la tabla cuentas_pyg_IBEX35
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
for index, row in df_05_cuentas_pyg.iterrows():
    cur.execute("""
    INSERT INTO "Cuentas_PYG_IBEX35" ("Ticker", "Año", "Ventas", "Beneficio_neto")
    VALUES (%s, %s, %s, %s);
    """, (row['ticker'], row['Año'], row['totalRevenue'], row['netIncome']))
    #Commit los cambios
    conn.commit()


cur.close()
conn.close()

In [20]:
#comprobamos que se han cargado los datos en la tabla
#Conectamos a la base de datos
conn= ps.connect(
    dbname= 'IBEX35_Análisis',
    user= 'postgres',
    password= 'admin',
    host= 'localhost',
    port= '5432'
)
#Creamos un cursor para ejecutar las consultas
cur = conn.cursor()
cur.execute("""
SELECT* FROM "Cuentas_PYG_IBEX35";
            """)
df_cuentas_pyg = pd.DataFrame(cur.fetchall(), columns=["Ticker", "Año", "Ventas", "Beneficio_neto"])
#Mostramos los resultados
df_cuentas_pyg

,Ticker,Año,Ventas,Beneficio_neto
0,ACS.MC,2020,3.493736e+10,8.960500e+08
1,ACS.MC,2021,2.783666e+10,-7.318700e+08
2,ACS.MC,2022,3.361523e+10,6.682270e+08
3,ACS.MC,2023,3.573776e+10,7.801230e+08
4,ACS.MC,2024,4.163312e+10,8.275800e+08
...,...,...,...,...
171,UNI.MC,2020,1.033204e+09,7.783100e+07
172,UNI.MC,2021,1.247682e+09,1.113202e+09
173,UNI.MC,2022,1.754926e+09,2.775760e+08
174,UNI.MC,2023,1.977103e+09,2.667030e+08


ya tendríamos todos los datos cargados en la base de datos, por lo que se podría pasar al EDA.
